In [2]:
import os
import json
import requests
from azure.cosmos import CosmosClient, PartitionKey, exceptions
from datetime import datetime as dt
from windows_tools.installed_software import get_installed_software

# Configuración de variables
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://andresdia02.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'ovpWKq3QnVf76g2KKnwQb2FArJz9X6lroZ3bqhvqXIkdE3rgxObPWRVWxNU6yWEwDxER5Imd983HACDbCIbnUQ=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'base1'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'container2')
}

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

def create_cosmos_client():
    return CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)

def create_database(client):
    try:
        return client.create_database(id=DATABASE_ID)
    except exceptions.CosmosResourceExistsError:
        return client.get_database_client(DATABASE_ID)

def create_container(db_client):
    try:
        return db_client.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
    except exceptions.CosmosResourceExistsError:
        return db_client.get_container_client(CONTAINER_ID)

def search_vulnerabilities(software, max_vulnerabilities=3):
    url = "https://vulners.com/api/v3/search/lucene/"
    headers = {"Content-Type": "application/json"}
    data = {
        "query": software,
        "size": max_vulnerabilities
    }
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            return response.json()
    except Exception as e:
        print(f"Error al buscar vulnerabilidades para {software}: {e}")
    return None

def get_cve_details(cve):
    url = f'https://cve.circl.lu/api/cve/{cve}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
    except Exception as e:
        print(f"Error al obtener detalles de CVE {cve}: {e}")
    return None

def format_sec_data(unique_id, json_data):
    unique_suffix = dt.now().strftime('%Y%m%d%H%M%S%f')
    sec_data = {
        'id': f"{unique_id}_{unique_suffix}",
        'partitionKey': 'Account1',
        'date': dt.now().strftime('%Y-%m-%d %H:%M:%S'),
        'vulnerability': json_data,
        'Description': json_data
    }
    return sec_data

def main():
    client = create_cosmos_client()
    db = create_database(client)
    container = create_container(db)

    installed_programs = [program['name'] for program in get_installed_software()]
    for program in installed_programs:
        print(f"Programa: {program}")
        vulnerabilities = search_vulnerabilities(program)
        if vulnerabilities and 'data' in vulnerabilities:
            for idx, vulnerability in enumerate(vulnerabilities['data']['search'], 1):
                if idx > 3:
                    break
                cve = vulnerability['_source']['id']
                cve_details = get_cve_details(cve)
                if cve_details:
                    print(f"\nVulnerabilidad {idx}:")
                    print(f"CVE: {cve}")
                    print(f"Resumen: {cve_details.get('summary', '')}")
                    print(f"Publicado: {cve_details.get('Published', '')}")
                    print(f"Última modificación: {cve_details.get('last-modified', '')}")
                    print(f"CVSS: {cve_details.get('cvss', '')}")

                    sec_data = format_sec_data(cve, cve_details)
                    container.create_item(body=sec_data)
        else:
            print("No se pudo obtener información sobre las vulnerabilidades.")
        print("\n")

if __name__ == "__main__":
    main()


Programa: StarUML 6.0.1


Programa: Cisco Packet Tracer 7.3.1 64Bit


Programa: GDR 1105 para SQL Server 2022 (KB5029379) (64-bit)


Programa: GDR 1110 para SQL Server 2022 (KB5032968) (64-bit)


Programa: Microsoft Azure Compute Emulator - v2.9.7


Programa: Microsoft SQL Server 2022 (64-bit)


Programa: Microsoft SQL Server 2022 (64-bit)


Programa: Mozilla Firefox (x64 es-ES)


Programa: Mozilla Maintenance Service


Programa: MPLAB XC8 C Compiler


Programa: 


Programa: 


Programa: Apache NetBeans IDE 12.5


Programa: Notepad++ (64-bit x64)


Programa: Tracker

Vulnerabilidad 2:
CVE: CVE-2013-3525
Resumen: SQL injection vulnerability in Approvals/ in Request Tracker (RT) 4.0.10 and earlier allows remote attackers to execute arbitrary SQL commands via the ShowPending parameter.  NOTE: the vendor disputes this issue, stating "We were unable to replicate it, and the individual that reported it retracted their report," and "we had verified that the claimed exploit did not function ac